In [2]:
### Stuff LCEL Chain ###
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader 
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    temperature=0.1
)

memory = ConversationBufferMemory() #return_messages=True

cache_dir = LocalFileStore("./.cache2/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,  
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs,embeddings) 

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant. Answer question using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"),
    ("human","{question}"),
])

memory.load_memory_variables({})

def ask_and_save(inputs):
    print(inputs.content)
    #content = inputs["content"]
    #question = inputs["question"]
    memory.save_context({"input":RunnablePassthrough()},{"output":inputs.content})


chain = {"context":retriever, "question":RunnablePassthrough()} | prompt | llm | RunnableLambda(ask_and_save)

chain.invoke("Is Aaronson guilty?")
chain.invoke("What message did he write in the table?")
chain.invoke("Who is Julia?")

According to the document, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.


ValidationError: 2 validation errors for HumanMessage
content
  str type expected (type=type_error.str)
content
  value is not a valid list (type=type_error.list)